In [3]:
import pyaudio
import wave
import openai
import numpy as np
import webrtcvad

def recording():
    CHUNK = 160
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 16000
    SILENCE_LIMIT = 2  # Time of silence needed to stop recording (seconds)
    WAVE_OUTPUT_FILENAME = "input.wav"

    vad = webrtcvad.Vad()
    vad.set_mode(3)
    
    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []
    silent_count = 0
    
    while True:
        data = stream.read(CHUNK)
        
        
        # Check for speech
        decoded_data = np.frombuffer(data, dtype=np.int16)
        audio_level = np.max(decoded_data)

        if  vad.is_speech(data, RATE):
            silent_count = 0
            frames.append(data)
        else:
            silent_count += 1
            
        # Check if there's been enough silence to stop recording
        if silent_count > SILENCE_LIMIT * RATE / CHUNK:
            break

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()
 
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

recording()

audio_file= open("input.wav", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
print(transcript)

Recording...
Finished recording.
{
  "text": "Hello, this is a test file testing if the current script actually is only appending the data to the frames list when only speech is present, omitting any breaks in between. Let's hope for the best."
}


In [2]:
import pyaudio
import wave
import openai
import numpy as np

def recording():
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    THRESHOLD = 1000  # Adjust the threshold according to your microphone sensitivity
    SILENCE_LIMIT = 3  # Time of silence needed to stop recording (seconds)
    WAVE_OUTPUT_FILENAME = "input.wav"

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    frames = []
    silent_count = 0
    
    while True:
        data = stream.read(CHUNK)
        frames.append(data)
        
        # Check audio level
        decoded_data = np.frombuffer(data, dtype=np.int16)
        audio_level = np.max(decoded_data)

        if audio_level > THRESHOLD:
            silent_count = 0
        else:
            silent_count += 1

        # Check if there's been enough silence to stop recording
        if silent_count > SILENCE_LIMIT * RATE / CHUNK:
            break

    print("Finished recording.")

    stream.stop_stream()
    stream.close()
    audio.terminate()
 
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

recording()

audio_file= open("input.wav", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)

Recording...
Finished recording.


In [6]:
print(transcript)

{
  "text": "[(\ube14\ub799\ud551\ud06c\uc758 \uc544\uc774\ucee8\ud0dd)] [(\ube14\ub799\ud551\ud06c\uc758 \uc544\uc774\ucee8\ud0dd)] [(\ube14\ub799\ud551\ud06c\uc758 \uc544\uc774\ucee8\ud0dd)]"
}
